# This notebook explores the initial scripts necessary to extract the financial data from Apple's SEC Filings

In [1]:
import pandas as pd

file_to_inspect = 'data/2024-10q.xls'

try:
    # First, we open the Excel file
    xls = pd.ExcelFile(file_to_inspect)
    
    # Now, we can ask it for the names of all the sheets inside
    print(f" Successfully opened {file_to_inspect}.")
    print("Here are the names of all the sheets found inside:")
    print(xls.sheet_names)

except Exception as e:
    print(f"❌ An error occurred: {e}")

 Successfully opened data/2024-10q.xls.
Here are the names of all the sheets found inside:
['Table_Of_Contents', 'TABLE1', 'TABLE2', 'TABLE3', 'INCOME_STATEMENT', 'TABLE4', 'BALANCE_SHEET', 'BALANCE_SHEET2', 'STOCKHOLDERS_EQUITY', 'CASH_FLOW', 'TABLE5', 'TABLE6', 'TABLE7', 'TABLE8', 'TABLE9', 'TABLE10', 'TABLE11', 'TABLE12', 'TABLE13', 'TABLE14', 'TABLE15', 'TABLE16', 'TABLE17', 'TABLE18', 'TABLE19', 'TABLE20', 'TABLE21', 'TABLE22', 'TABLE23', 'TABLE24']
